## Ejercicio 1: Crear un entorno para un Robot Pick&Place

Comenzamos importando las librerías necesarias.

In [1]:
%matplotlib qt

import gymnasium
from gymnasium import spaces
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

## Creación del entorno

### Sistemas de recompensas

Establecemos 3 sistemas distintos de recompensa:
- Recompensa simple por objetivos alcanzados.
- Recompensa gradual por acercarse al objetivo.
- Recompensa mixta (combinación de hitos y distancias).

In [2]:
def calculate_reward_simple(env):
    """
    Calcula la recompensa basada en objetivos alcanzados y penaliza soltar la pieza fuera del objetivo.

    Args:
        env (Environment): El entorno del robot (posiciones y estados).

    Returns:
        float: Recompensa acumulada según el estado actual.
    """
    reward = -0.01  # Penalización por cada paso realizado.

    # Recompensa por recoger la pieza.
    if env.has_piece == 1 and env.piece_x_position == -1 and not hasattr(env, 'piece_collected'):
        reward += 50
        setattr(env, 'piece_collected', True)

    # Recompensa por llevar la pieza al objetivo.
    if (env.piece_x_position == env.goal_x_position and
        env.piece_y_position == env.goal_y_position and
        env.piece_z_position == env.goal_z_position and
        not hasattr(env, 'goal_reached')):
        reward += 70
        setattr(env, 'goal_reached', True)

    # Penalización por soltar la pieza en una posición incorrecta.
    if env.has_piece == 0 and not hasattr(env, 'piece_placed') and (
        env.piece_x_position != env.goal_x_position or
        env.piece_y_position != env.goal_y_position or
        env.piece_z_position != env.goal_z_position
    ):
        reward -= 30  # Penalización significativa.
        setattr(env, 'piece_placed', True)

    # Penalización por movimientos redundantes.
    if hasattr(env, 'visited_positions'):
        current_position = (env.robot_x_position, env.robot_y_position, env.robot_z_position)
        if current_position in env.visited_positions:
            reward -= 0.01
        else:
            env.visited_positions.add(current_position)
    else:
        env.visited_positions = set([(env.robot_x_position, env.robot_y_position, env.robot_z_position)])

    return reward

def calculate_reward_distance(env):
    """
    Calcula la recompensa basada en la distancia al objetivo y los objetivos alcanzados,
    penalizando además soltar la pieza en una posición incorrecta.

    Args:
        env (Environment): El entorno que contiene el estado actual del robot. 

    Returns:
        float: La recompensa calculada para el estado actual del entorno.
    """
    reward = 0  # Penalización inicial.

    if env.has_piece == 0:
        # Penalización proporcional a la distancia al objeto.
        distance_to_piece = ((env.robot_x_position - env.piece_x_position) ** 2 +
                             (env.robot_y_position - env.piece_y_position) ** 2 +
                             (env.robot_z_position - env.piece_z_position) ** 2) ** 0.5
        reward += distance_to_piece * 2
    else:
        # Penalización proporcional a la distancia al objetivo.
        distance_to_goal = ((env.robot_x_position - env.goal_x_position) ** 2 +
                            (env.robot_y_position - env.goal_y_position) ** 2 +
                            (env.robot_z_position - env.goal_z_position) ** 2) ** 0.5
        reward += distance_to_goal * 2

    # Recompensa por recoger la pieza.
    if env.has_piece == 1 and not hasattr(env, 'piece_collected'):
        reward += 50
        setattr(env, 'piece_collected', True)

    # Recompensa por alcanzar el objetivo.
    if (env.piece_x_position == env.goal_x_position and
        env.piece_y_position == env.goal_y_position and
        env.piece_z_position == env.goal_z_position and
        not hasattr(env, 'goal_reached')):
        reward += 70
        setattr(env, 'goal_reached', True)

    # Penalización por soltar la pieza en una posición incorrecta.
    if env.has_piece == 0 and not hasattr(env, 'piece_placed') and (
        env.piece_x_position != env.goal_x_position or
        env.piece_y_position != env.goal_y_position or
        env.piece_z_position != env.goal_z_position
    ):
        reward -= 30
        setattr(env, 'piece_placed', True)

    # Penalización por movimientos redundantes.
    if hasattr(env, 'visited_positions'):
        current_position = (env.robot_x_position, env.robot_y_position, env.robot_z_position)
        if current_position in env.visited_positions:
            reward -= 7
        else:
            env.visited_positions.add(current_position)
    else:
        env.visited_positions = set([(env.robot_x_position, env.robot_y_position, env.robot_z_position)])

    return reward

def calculate_reward_mixed(env):
    """
    Calcula la recompensa mixta basada en objetivos alcanzados y distancia al objetivo,
    penalizando además soltar la pieza en una posición incorrecta.

    Args:
        env (Environment): El entorno que contiene el estado actual del robot.

    Returns:
        float: La recompensa calculada para el estado actual del entorno.
    """
    reward = -0.25  # Penalización por cada paso realizado.

    # Penalización basada en la distancia.
    if env.has_piece == 0:
        distance_to_piece = ((env.robot_x_position - env.piece_x_position) ** 2 +
                             (env.robot_y_position - env.piece_y_position) ** 2 +
                             (env.robot_z_position - env.piece_z_position) ** 2) ** 0.5
        reward += distance_to_piece * 1.75
    else:
        distance_to_goal = ((env.robot_x_position - env.goal_x_position) ** 2 +
                            (env.robot_y_position - env.goal_y_position) ** 2 +
                            (env.robot_z_position - env.goal_z_position) ** 2) ** 0.5
        reward += distance_to_goal * 1.75

    # Recompensa por recoger la pieza.
    if env.has_piece == 1 and not hasattr(env, 'piece_collected'):
        reward += 50
        setattr(env, 'piece_collected', True)

    # Recompensa por alcanzar el objetivo.
    if (env.piece_x_position == env.goal_x_position and
        env.piece_y_position == env.goal_y_position and
        env.piece_z_position == env.goal_z_position and
        not hasattr(env, 'goal_reached')):
        reward += 70
        setattr(env, 'goal_reached', True)

    # Penalización por soltar la pieza en una posición incorrecta.
    if env.has_piece == 0 and not hasattr(env, 'piece_placed') and (
        env.piece_x_position != env.goal_x_position or
        env.piece_y_position != env.goal_y_position or
        env.piece_z_position != env.goal_z_position
    ):
        reward -= 30  # Penalización significativa.
        setattr(env, 'piece_placed', True)

    # Penalización por movimientos redundantes.
    if hasattr(env, 'visited_positions'):
        current_position = (env.robot_x_position, env.robot_y_position, env.robot_z_position)
        if current_position in env.visited_positions:
            reward -= 6
        else:
            env.visited_positions.add(current_position)
    else:
        env.visited_positions = set([(env.robot_x_position, env.robot_y_position, env.robot_z_position)])

    return reward

### Definición del entorno

Para crear el entorno, creamos una clase. 
Dentro de ella, definimos el constructor, donde definimos el espacio de acciones y de observaciones. 
También, inicializamos el estado, creando las condiciones iniciales, y la visualización del robot.

Podemos decir que generamos nuestros objetos, el robot y la pieza, al definir sus posiciones; y establecemos el objetivo: la posición a la que el robot debe llevar la pieza.

Escogemos un sistema de recompensas. Al actuar de forma aleatoria, la configuración de este sistema no va a cambiar cómo actúa nuestro robot, pero sí es útil de cara a futuras implementaciones de algoritmos.

In [3]:
class RobotPickAndPlaceEnv(gymnasium.Env):
    """
    Representa el entorno del robot para resolver el problema de pick & place.

    Este entorno simula un espacio discreto en 3D donde un robot debe recoger una 
    pieza en un punto inicial y transportarla a un objetivo. El robot puede moverse 
    en las direcciones X, Y y Z, además de abrir o cerrar su pinza para coger y soltar la pieza.

    Attributes:
        robot_position: Posición actual del robot en (x, y, z).
        piece_position: Posición actual de la pieza en (x, y, z). (-1, -1, -1 si está siendo transportada).
        goal_position: Posición del objetivo en (x, y, z).
        has_piece (bool): Indica si el robot tiene la pieza agarrada (1 si tiene la pieza, 0 en caso contrario).
        step_limit (int): Número máximo de pasos permitidos.
        action_space (gymnasium.spaces.Discrete): Espacio de acciones (0-6).
        observation_space (gymnasium.spaces.Tuple): Espacio de observaciones del entorno.
        reward_function: Función de recompensa utilizada para evaluar los pasos del robot.
    """
    def __init__(self):
        """
        Inicializa el entorno del robot.

        Configura los espacios de acción y observación, y define las posiciones iniciales 
        del robot, la pieza y el objetivo.
        """
        super(RobotPickAndPlaceEnv, self).__init__()

        # Definir espacio de acciones: 0 = mover izquierda, 1 = mover derecha, 2 = mover abajo, 3 = mover arriba, 4 = mover atrás, 5 = mover delante, 6 = abrir/cerrar pinza
        self.action_space = spaces.Discrete(7)

        # Definir espacio de observaciones
        self.observation_space = spaces.Tuple((
            spaces.Discrete(10),  # Posición del robot en el eje X (0-9)
            spaces.Discrete(10),  # Posición del robot en el eje Y (0-9)
            spaces.Discrete(10),  # Posición del robot en el eje Z (0-9)
            spaces.Discrete(2),   # Tiene pieza (0 o 1)
            spaces.Discrete(11),  # Posición de la pieza en el eje X (0-9) (-1 si está siendo transportada)
            spaces.Discrete(11),  # Posición de la pieza en el eje Y (0-9) (-1 si está siendo transportada)
            spaces.Discrete(11)   # Posición de la pieza en el eje Z (0-9) (-1 si está siendo transportada)
        ))

        # Inicializar estado
        self.reset()

        # Inicializar figura, eje y gráfica de matplotlib
        self.fig = plt.figure()
        self.ax = self.fig.add_subplot(111, projection='3d')
        self.ax.set_xlim(0, 10)
        self.ax.set_ylim(0, 10)
        self.ax.set_zlim(0, 10)
        self.ax.set_xlabel('X')
        self.ax.set_ylabel('Y')
        self.ax.set_zlabel('Z')

        self.robot_plot, = self.ax.plot([self.robot_x_position], [self.robot_y_position], [self.robot_z_position], 'ro', label="Robot")
        self.piece_plot, = self.ax.plot([self.piece_x_position], [self.piece_y_position], [self.piece_z_position], 'bo', label="Pieza")
        self.goal_plot, = self.ax.plot([self.goal_x_position], [self.goal_y_position], [self.goal_z_position], 'go', label="Objetivo")
        self.ax.legend()

    def reset(self):
        """
        Reinicia el entorno a su estado inicial.

        Returns:
            tuple: Observación inicial del entorno.
        """
        # Condiciones iniciales
        self.robot_x_position = 0 # El robot comienza en la posición (0,0,0)
        self.robot_y_position = 0
        self.robot_z_position = 0
        self.has_piece = 0  # El robot no tiene la pieza
        self.piece_x_position = 5  # La pieza comienza en la posición (5,5,5)
        self.piece_y_position = 5
        self.piece_z_position = 5
        self.goal_x_position = 9  # El objetivo está en la posición (9, 7, 9)
        self.goal_y_position = 7
        self.goal_z_position = 9

        return self._get_observation()

    def step(self, action):
        """
        Realiza un paso en el entorno según la acción proporcionada.

        Args:
            action (int): Índice de la acción a realizar (0-6).

        Returns:
            tuple: Estado actual del entorno, recompensa obtenida, 
                indicador de si la tarea ha terminado, indicador de truncado, 
                y diccionario de información adicional.
        """
        # Asegurarse de que la acción es válida
        assert self.action_space.contains(action)

        done = False
        truncated = False

        # Realizar movimientos según la acción
        if action == 0:  # Mover izquierda
            self.robot_x_position = max(0, self.robot_x_position - 1)
        elif action == 1:  # Mover derecha
            self.robot_x_position = min(9, self.robot_x_position + 1)
        elif action == 2:  # Mover abajo
            self.robot_y_position = max(0, self.robot_y_position - 1)
        elif action == 3:  # Mover arriba
            self.robot_y_position = min(9, self.robot_y_position + 1)
        elif action == 4:  # Mover atrás
            self.robot_z_position = max(0, self.robot_z_position - 1)
        elif action == 5:  # Mover adelante
            self.robot_z_position = min(9, self.robot_z_position + 1)
        elif action == 6:  # Abrir/cerrar pinza
            if not self.has_piece:
                if (self.robot_x_position == self.piece_x_position and
                    self.robot_y_position == self.piece_y_position and
                    self.robot_z_position == self.piece_z_position):
                    print("¡La pieza ha sido recogida!")
                    self.has_piece = 1
                    self.piece_x_position = -1
                    self.piece_y_position = -1
                    self.piece_z_position = -1
                else:
                    print("No hay pieza en esta posición.")
            else:
                self.has_piece = 0
                self.piece_x_position = self.robot_x_position
                self.piece_y_position = self.robot_y_position
                self.piece_z_position = self.robot_z_position
                if (self.piece_x_position == self.goal_x_position and
                    self.piece_y_position == self.goal_y_position and
                    self.piece_z_position == self.goal_z_position):
                    print("¡La pieza está en el objetivo!")
                    done = True

        # Calcular la recompensa usando la función de recompensa seleccionada
        if hasattr(self, 'reward_function') and callable(self.reward_function):
            reward = self.reward_function(self)
        else:
            raise ValueError("La función de recompensa no está definida o no es válida.")

        # Mostrar el estado del robot y pieza
        print(f"Robot: ({self.robot_x_position}, {self.robot_y_position}, {self.robot_z_position}) | "
            f"Pieza: ({self.piece_x_position}, {self.piece_y_position}, {self.piece_z_position}) | "
            f"Has_piece: {self.has_piece}")

        return self._get_observation(), reward, done, truncated, {}



    def render(self, mode='human'):
        """
        Renderiza el estado actual del entorno en una gráfica 3D.
        """
        # Actualizar posición del robot
        self.robot_plot.set_data([self.robot_x_position], [self.robot_y_position])
        self.robot_plot.set_3d_properties([self.robot_z_position])

        # Actualizar posición de la pieza
        if self.has_piece:
            self.piece_plot.set_data([self.robot_x_position], [self.robot_y_position])
            self.piece_plot.set_3d_properties([self.robot_z_position])
        else:
            self.piece_plot.set_data([self.piece_x_position], [self.piece_y_position])
            self.piece_plot.set_3d_properties([self.piece_z_position])

        plt.pause(0.25)

    def _get_observation(self):
        """
        Obtiene la observación actual del entorno.

        Returns:
            tuple: Estado actual del entorno.
        """
        return (self.robot_x_position, self.robot_y_position, self.robot_z_position, self.has_piece, self.piece_x_position, self.piece_y_position, self.piece_z_position)

    def close(self):
        """
        Cierra la visualización del entorno.
        """
        plt.close()

### Análisis dimensional

- Tenemos 7 espacios de acciones posibles.

- (1000 (posiciones del robot) × 2 (pinza) × 1001 (posiciones de la pieza)) = 2.002.000 estados posibles.

- 2.002.000×7 = 14.014.000 combinaciones estado-acción.

## Ejecución del entorno

Inicializamos y reiniciamos el entorno, y seguimos una trayectoria óptima y otra subóptima para ver como funciona cada sistema de recompensas.

In [4]:
if __name__ == "__main__":
    # Inicializamos el entorno
    env = RobotPickAndPlaceEnv()

    optimal_trajectory = [
        (1, 0),  # Mover +X
        (1, 0),  # Mover +X
        (1, 0),  # Mover +X
        (1, 0),  # Mover +X
        (1, 0),  # Mover +X
        (3, 0),  # Mover +Y
        (3, 0),  # Mover +Y
        (3, 0),  # Mover +Y
        (3, 0),  # Mover +Y
        (3, 0),  # Mover +Y
        (5, 0),  # Mover +Z
        (5, 0),  # Mover +Z
        (5, 0),  # Mover +Z
        (5, 0),  # Mover +Z
        (5, 0),  # Mover +Z
        (6, 0),  # Cerrar pinza
        (5, 0),  # Mover +Z
        (5, 0),  # Mover +Z
        (5, 0),  # Mover +Z
        (5, 0),  # Mover +Z
        (1, 0),  # Mover +X
        (1, 0),  # Mover +X
        (1, 0),  # Mover +X
        (1, 0),  # Mover +X
        (3, 0),  # Mover +Y
        (3, 0),  # Mover +Y
        (6, 0)   # Abrir pinza
    ]

    suboptimal_trajectory = [
        (1, 0),  # Mover +X
        (1, 0),  # Mover +X
        (1, 0),  # Mover +X
        (1, 0),  # Mover +X
        (3, 0),  # Mover +Y
        (3, 0),  # Mover +Y
        (3, 0),  # Mover +Y
        (3, 0),  # Mover +Y
        (3, 0),  # Mover +Y
        (5, 0),  # Mover +Z
        (5, 0),  # Mover +Z
        (5, 0),  # Mover +Z
        (4, 0),  # Mover -Y (innecesario)
        (1, 0),  # Mover +X
        (5, 0),  # Mover +Z
        (5, 0),  # Mover +Z
        (5, 0),  # Mover +Z (innecesario)
        (6, 0),  # Abrir pinza
        (5, 0),  # Mover +Z (innecesario)
        (5, 0),  # Mover +Z (innecesario)
        (4, 0),  # Mover -Y (innecesario)
        (4, 0),  # Mover -Y (innecesario)
        (5, 0),  # Mover +Z (innecesario)
        (3, 0),  # Mover +Y (innecesario)
        (1, 0),  # Mover +X
        (1, 0),  # Mover +X
        (3, 0),  # Mover +Y
        (3, 0),  # Mover +Y
        (1, 0),  # Mover +X
        (1, 0),  # Mover +X
        (6, 0),  # Abrir pinza (innecesario)
        (2, 0),  # Mover -Y (innecesario)
        (3, 0),  # Mover +Y (innecesario)
        (6, 0),  # Cerrar pinza (innecesario)
        (4, 0),  # Mover -Z (innecesario)
        (5, 0),  # Mover +Z (innecesario)
        (1, 0),  # Mover +X
        (5, 0),  # Mover +Z
        (5, 0),  # Mover +Z
        (5, 0),  # Mover +Z
        (5, 0),  # Mover +Z
        (2, 0),  # Mover -Y
        (6, 0)   # Abrir pinza
    ]


    # Funciones de recompensa a probar
    reward_functions = [
        ("Simple", calculate_reward_simple),
        ("Distance", calculate_reward_distance),
        ("Mixed", calculate_reward_mixed)
    ]

    for name, reward_function in reward_functions:
        env.reward_function = reward_function

        print(f"\nEvaluando función de recompensa: {name}")

        # Evaluar trayectoria óptima
        print("\nTrayectoria óptima:")
        env.reset()
        optimal_reward = 0
        for action, _ in optimal_trajectory:
            env.render()  # Mostrar visualización en cada paso
            _, reward, done, _, _ = env.step(action)
            optimal_reward += reward
            if done:
                break
        print(f"Recompensa total acumulada: {optimal_reward}")

        # Evaluar trayectoria subóptima
        print("\nTrayectoria subóptima:")
        env.reset()
        suboptimal_reward = 0
        for action, _ in suboptimal_trajectory:
            env.render()  # Mostrar visualización en cada paso
            _, reward, done, _, _ = env.step(action)
            suboptimal_reward += reward
            if done:
                break
        print(f"Recompensa total acumulada: {suboptimal_reward}")

    env.close()


Evaluando función de recompensa: Simple

Trayectoria óptima:


2025-01-04 14:45:24.334 python[11982:1135714] +[IMKClient subclass]: chose IMKClient_Modern
2025-01-04 14:45:24.334 python[11982:1135714] +[IMKInputSession subclass]: chose IMKInputSession_Modern


Robot: (1, 0, 0) | Pieza: (5, 5, 5) | Has_piece: 0
Robot: (2, 0, 0) | Pieza: (5, 5, 5) | Has_piece: 0
Robot: (3, 0, 0) | Pieza: (5, 5, 5) | Has_piece: 0
Robot: (4, 0, 0) | Pieza: (5, 5, 5) | Has_piece: 0
Robot: (5, 0, 0) | Pieza: (5, 5, 5) | Has_piece: 0
Robot: (5, 1, 0) | Pieza: (5, 5, 5) | Has_piece: 0
Robot: (5, 2, 0) | Pieza: (5, 5, 5) | Has_piece: 0
Robot: (5, 3, 0) | Pieza: (5, 5, 5) | Has_piece: 0
Robot: (5, 4, 0) | Pieza: (5, 5, 5) | Has_piece: 0
Robot: (5, 5, 0) | Pieza: (5, 5, 5) | Has_piece: 0
Robot: (5, 5, 1) | Pieza: (5, 5, 5) | Has_piece: 0
Robot: (5, 5, 2) | Pieza: (5, 5, 5) | Has_piece: 0
Robot: (5, 5, 3) | Pieza: (5, 5, 5) | Has_piece: 0
Robot: (5, 5, 4) | Pieza: (5, 5, 5) | Has_piece: 0
Robot: (5, 5, 5) | Pieza: (5, 5, 5) | Has_piece: 0
¡La pieza ha sido recogida!
Robot: (5, 5, 5) | Pieza: (-1, -1, -1) | Has_piece: 1
Robot: (5, 5, 6) | Pieza: (-1, -1, -1) | Has_piece: 1
Robot: (5, 5, 7) | Pieza: (-1, -1, -1) | Has_piece: 1
Robot: (5, 5, 8) | Pieza: (-1, -1, -1) | Has_

## Conclusiones

- Función de Recompensa Simple: Esta función valora de manera significativa el progreso hacia los objetivos y permite un margen de exploración porque las penalizaciones son menos severas. Podría penalizar más las trayectorias menos óptimas.

- Función de Recompensa Distancia: Esta función penaliza la distancia aunque no es muy agresiva. Se podría aumentar la penalización por movimientos redundantes.

- Función de Recompensa Mixta: Esta función ofrece un buen balance entre objetivos y distancia. Al igual que en la Simple, podría penalizar más las trayectorias menos óptimas.